##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classificação de Imagens

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver em TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Código Fonte no GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Baixar notebook</a>
  </td>
</table>

Este tutorial mostra como classificar gatos ou cães a partir de imagens. Ele constrói um classificador de imagens usando um modelo `tf.keras.Sequential` e carrega dados usando `tf.keras.preprocessing.image.ImageDataGenerator`. Você obterá alguma experiência prática e desenvolverá a intuição para os seguintes conceitos:

* Criando pipelines de entrada de dados_ usando a classe `tf.keras.preprocessing.image.ImageDataGenerator` para trabalhar com eficiência com os dados no disco para usar com o modelo.
* _Overfitting_ - Como identificar e evitá-lo.
* _Aumento de dados_ e _dropout_ - Técnicas-chave para combater o overfitting em tarefas de visão computacional para incorporar no pipeline de dados e no modelo classificador de imagens.

Este tutorial segue um fluxo de trabalho básico de aprendizado de máquina:

1. Examine e entenda os dados
2. Construa um pipeline de entrada
3. Construa o modelo
4. Treine o modelo
5. Teste o modelo
6. Melhore o modelo e repita o processo

## Importar pacotes

Vamos começar importando os pacotes necessários. O pacote `os` é usado para ler arquivos e estrutura de diretórios, o NumPy é usado para converter a lista python em um array numpy e executar operações matriciais necessárias e `matplotlib.pyplot` para plotar o gráfico e exibir imagens nos dados de treinamento e validação.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

Importe o Tensorflow e as classes Keras necessárias para construir nosso modelo.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

## Carregando Dados

Comece baixando o conjunto de dados. Este tutorial usa a versão filtrada do conjunto de dados <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a> do Kaggle. 

Baixe a versão de arquivo do conjunto de dados e armazene-a no diretório "/tmp/".

In [0]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

O conjunto de dados tem a seguinte estrutura de diretórios:

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

Após extrair seu conteúdo, atribua variáveis com o caminho de arquivo adequado para o conjunto de treinamento e validação.

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_cats_dir = os.path.join(train_dir, 'cats')  # diretório com as imagens de gato para treinamento
train_dogs_dir = os.path.join(train_dir, 'dogs')  # diretório com as imagens de cães para treinamento
validation_cats_dir = os.path.join(validation_dir, 'cats')  # diretório com as imagens de gatos para validação
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # diretório com as imagens de cães para validação

### Entendendo os Dados

Vejamos quantas imagens de cães e gatos estão nos diretórios de treinamento e validação:

In [0]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [0]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

Por conveniência, configure variáveis a serem usadas durante o pré-processamento do conjunto de dados e o treinamento da rede.

In [0]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

## Preparação dos Dados

Formate as imagens em tensores de ponto flutuante pré-processados adequadamente antes de alimentar a rede:

1. Leia imagens do disco.
2. Decodifique o conteúdo dessas imagens e converta-o no formato de grade apropriado, conforme o conteúdo RGB.
3. Converta-os em tensores de ponto flutuante.
4. Redimensione os tensores de valores entre 0 e 255 para valores entre 0 e 1, pois as redes neurais preferem lidar com pequenos valores de entrada.

Felizmente, todas essas tarefas podem ser realizadas com a classe `ImageDataGenerator` fornecida pelo `tf.keras`. Ele pode ler imagens do disco e pré-processá-las em tensores adequados. Ele também configurará geradores que convertem essas imagens em lotes de tensores - úteis ao treinar a rede.

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Gerador para nossos dados de treinamento
validation_image_generator = ImageDataGenerator(rescale=1./255) # Gerador para nossos dados de validação

Depois de definir os geradores para imagens de treinamento e validação, o método `flow_from_directory` carrega imagens do disco e redimensiona as imagens nas dimensões necessárias.

In [0]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

### Visualizando as imagens de treinamento

Visualize as imagens de treinamento extraindo um lote de imagens do gerador de treinamento - que são 32 imagens neste exemplo - e plote cinco delas com `matplotlib`.

In [0]:
sample_training_images, _ = next(train_data_gen)

A função `next` retorna um lote do conjunto de dados. O valor de retorno da função `next` está na forma de `((x_train, y_train)` onde x_train está treinando características e y_train, seus rótulos. Descarte os rótulos para visualizar apenas as imagens de treinamento.

In [0]:
# Esta função plotará imagens na forma de uma tabela com 1 linha e 5 colunas onde as imagens são colocadas em cada coluna.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
plotImages(sample_training_images[:5])

## Criando o Modelo

O modelo consiste em três blocos de convolução com uma camada `max pool` em cada um deles. Há uma camada totalmente conectada com 512 unidades em cima, ativada por uma função de ativação `relu`. O modelo gera probabilidades de classe com base na classificação binária pela função de ativação `sigmoid`.

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

### Compilando o Modelo

Para esse tutorial, escolhemos o otimizador *ADAM* e a função de perda *binary cross entropy*. Para visualizar a acurácia do treinamento e validação  a cada época de treinamento, passamos o argumento `metrics`.


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Resumo do Modelo

Veja todas as camadas da rede usando o método `summary` do modelo:

In [0]:
model.summary()

### Treinando o Modelo

Use o método `fit_generator` da classe `ImageDataGenerator` para treinar a rede.

In [0]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Visualizando resultados do treinamento

Agora visualize os resultados depois de treinar a rede.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Como você pode ver nas plotagens, a acurácia do treinamento e a acurácia da validação são reduzidas por uma grande margem e o modelo alcançou apenas cerca de ** 70% ** de acurácia no conjunto de validação.

Vamos analisar o que deu errado e tentar aumentar o desempenho geral do modelo.

## Overfitting

Nas plotagens acima, a acurácia do treinamento aumenta linearmente ao longo do tempo, enquanto a acurácia da validação fica em torno de 70% no processo de treinamento. Além disso, a diferença de acurácia entre treinamento e validação é perceptível - um sinal de *overfitting*.

Quando há um pequeno número de exemplos de treinamento, o modelo algumas vezes aprende com ruídos ou detalhes indesejados dos exemplos de treinamento - a ponto de impactar negativamente o desempenho do modelo em novos exemplos. Esse fenômeno é conhecido como overfitting. Isso significa que o modelo está superajustado e terá dificuldade em generalizar um novo conjunto de dados.

Existem várias maneiras de combater o excesso de ajustes no processo de treinamento. Neste tutorial, você usará *data augmentation* e adicionará *dropout* ao nosso modelo.

## Data augmentation

O overfitting geralmente ocorre quando há um pequeno número de exemplos de treinamento. Uma maneira de corrigir esse problema é aumentar o conjunto de dados para que ele tenha um número suficiente de exemplos de treinamento. O aumento de dados adota a abordagem de gerar mais dados de treinamento a partir de amostras de treinamento existentes, aumentando as amostras usando transformações aleatórias que produzem imagens com aparência crível. O objetivo é que o modelo nunca veja exatamente a mesma imagem duas vezes durante o treinamento. Isso ajuda a expor o modelo a mais aspectos dos dados e a generalizar melhor.

Implemente isso no `tf.keras` usando a classe `ImageDataGenerator`. Passe diferentes transformações para o conjunto de dados e ele se encarregará de aplicá-lo durante o processo de treinamento.

### Aumentando e Visualizando os Dados

Comece aplicando rotações horizontais aleatórias ao conjunto de dados e veja como são as imagens individuais após a transformação.

### Aplicanco rotação horizontal

Passe `horizontal_flip` como argumento para a classe `ImageDataGenerator` e defina-o como `True` para aplicar esse aumento.

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

Pegue uma imagem de amostra dos exemplos de treinamento e repita-a cinco vezes para que o aumento seja aplicado à mesma imagem cinco vezes.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
# Reutilize a mesma função de plotagem personalizada definida e usada
# acima para visualizar as imagens de treinamento
plotImages(augmented_images)

### Rotacione a Imagem Aleatoriamente

Vamos dar uma olhada em um aumento diferente chamado rotação e aplicar 45 graus de rotação aleatoriamente nos exemplos de treinamento.

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Aplique aumento de zoom

Aplique um aumento de zoom ao conjunto de dados para ampliar imagens até 50% aleatoriamente.

In [0]:
# zoom_range de 0 - 1 onde 1 = 100%.
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5) # 

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Coloque tudo junto

Aplique todos os aumentos anteriores. Aqui, você aplicou redimensionamento, rotação de 45 graus, deslocamento de largura, deslocamento de altura, aumento horizontal de zoom e zoom nas imagens de treinamento.

In [0]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [0]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

Visualize como uma única imagem ficaria cinco vezes diferentes ao passar esses aprimoramentos aleatoriamente para o conjunto de dados.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Crie o gerador de dados de validação

Geralmente, aplique apenas o aumento de dados aos exemplos de treinamento. Nesse caso, apenas redimensione novamente as imagens de validação e as converta em lotes usando o `ImageDataGenerator`.

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [0]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## Dropout

Outra técnica para reduzir o overfitting é introduzir *dropout* na rede. É uma forma de *regularização* que força os pesos na rede a aceitar apenas valores pequenos, o que torna a distribuição dos valores de peso mais regular e a rede pode reduzir o excesso de ajustes em pequenos exemplos de treinamento. O abandono é uma das técnicas de regularização usadas neste tutorial

Quando você aplica a dropout a uma camada, ela remove aleatoriamente (definido como zero) o número de unidades de saída da camada aplicada durante o processo de treinamento. A desistência recebe um número fracionário como valor de entrada, na forma de 0.1, 0.2, 0.4 etc. Isso significa eliminar 10%, 20% ou 40% das unidades de saída aleatoriamente da camada aplicada.

Ao aplicar o abandono de 0.1 a uma determinada camada, ele mata aleatoriamente 10% das unidades de saída em cada época de treinamento.

Crie uma arquitetura de rede com esse novo recurso de eliminação e aplique-a a diferentes convoluções e camadas totalmente conectadas.

## Criando uma Nova Rede com Dropouts

Aqui, você aplica o dropout à primeira e à última camada máxima do pool. A aplicação de dropout definirá aleatoriamente 20% dos neurônios para zero durante cada época de treinamento. Isso ajuda a evitar ajustes excessivos no conjunto de dados de treinamento.

In [0]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

### Compilando o Modelo

Depois de introduzir desistências na rede, compile o modelo e visualize o resumo das camadas.

In [0]:
model_new.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_new.summary()

### Treinando o Modelo

Depois de introduzir com sucesso os aprimoramentos de dados nos exemplos de treinamento e adicionar dropout à rede, treine esta nova rede:

In [0]:
history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Visualizando o Modelo

Visualize o novo modelo após o treinamento, você pode ver que há muito menos ajuste do que antes. A acurácia deve aumentar após o treinamento do modelo por mais épocas.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()